### Deploy Resnest model

#### 1. Download Resnet model

In [1]:
mkdir /tmp/resnet

In [3]:
!curl -s https://storage.googleapis.com/download.tensorflow.org/models/official/20181001_resnet/savedmodels/resnet_v2_fp32_savedmodel_NHWC_jpg.tar.gz | tar --strip-components=2 -C /tmp/resnet -xvz

x 1538687457/
x 1538687457/variables/
x 1538687457/variables/variables.index
x 1538687457/variables/variables.data-00000-of-00001
x 1538687457/saved_model.pb


In [14]:
ls /tmp/resnet/

1538687457/


In [19]:
pwd

'/Users/macbook/Desktop/learn-machine-learning-in-two-months/deployment/distributed-tensorflow'

In [23]:
ls /tmp/resnet

1538687457/


#### 2. Kéo TF Serving Model về

In [25]:
!docker pull tensorflow/serving

Using default tag: latest
latest: Pulling from tensorflow/serving

39215d1d: Pulling fs layer 
88e9f672: Pulling fs layer 
3b1caad4: Pulling fs layer 
a26005b7: Pulling fs layer 
76a48494: Pulling fs layer 
79dc1d76: Pulling fs layer 
cc9ce8fa: Pulling fs layer 
Digest: sha256:f7e59a29cbc17a6b507751cddde37bccad4407c05ebf2c13b8e6ccb7d2e9affbA
Status: Downloaded newer image for tensorflow/serving:latest
docker.io/tensorflow/serving:latest


#### 3. Deploy lên Docker

Deploy và chạy API trên cổng **8501**

In [28]:
!docker run -p 8501:8501 --name tfserving_resnet \
--mount type=bind,source=/tmp/resnet,target=/models/resnet \
-e MODEL_NAME=resnet -t tensorflow/serving

2019-08-05 09:27:16.235457: I tensorflow_serving/model_servers/server.cc:82] Building single TensorFlow model file config:  model_name: resnet model_base_path: /models/resnet
2019-08-05 09:27:16.235812: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2019-08-05 09:27:16.235875: I tensorflow_serving/model_servers/server_core.cc:561]  (Re-)adding model: resnet
2019-08-05 09:27:16.348780: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: resnet version: 1538687457}
2019-08-05 09:27:16.348838: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: resnet version: 1538687457}
2019-08-05 09:27:16.348857: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: resnet version: 1538687457}
2019-08-05 09:27:16.349102: I external/org_tensorflow/tensorflow/contrib/session_bundle/bundle_shim.cc:363] Attempting to load native SavedModelBundle in bundle-shim

Giải thích dòng lệnh:

- **-p 8501:8501** : Cổng mà bạn serve API
- **--name** : Tên của container
- **--mount type=bind,source=/tmp/resnet,target=/models/resnet** : Nối docker container với thư mục đang chứa model
- **-e MODEL_NAME=resnet** : Tên của model
- **-t tensorflow/serving** : Chạy container trên image tensorflow/serving

#### 4. Viết hàm gọi API

In [33]:
from __future__ import print_function

import base64
import requests

# The server URL specifies the endpoint of your server running the ResNet
# model with the name "resnet" and using the predict interface.
SERVER_URL = 'http://localhost:8501/v1/models/resnet:predict'

# The image URL is the location of the image we should send to the server
IMAGE_URL = 'https://tensorflow.org/images/blogs/serving/cat.jpg'

def call_api():
  # Download the image
  dl_request = requests.get(IMAGE_URL, stream=True)
  dl_request.raise_for_status()

  # Compose a JSON Predict request (send JPEG image in base64).
  jpeg_bytes = base64.b64encode(dl_request.content).decode('utf-8')
  predict_request = '{"instances" : [{"b64": "%s"}]}' % jpeg_bytes

  # Send few requests to warm-up the model.
  for _ in range(3):
    response = requests.post(SERVER_URL, data=predict_request)
    response.raise_for_status()

  # Send few actual requests and report average latency.
  total_time = 0
  num_requests = 10
  for _ in range(num_requests):
    response = requests.post(SERVER_URL, data=predict_request)
    response.raise_for_status()
    total_time += response.elapsed.total_seconds()
    prediction = response.json()['predictions'][0]

  print('Prediction class: {}, avg latency: {} ms'.format(
      prediction['classes'], (total_time*1000)/num_requests))

In [38]:
print("Predicting...")
call_api()
print("Done...")

Predicting...
Prediction class: 286, avg latency: 130.5545 ms
Done...


#### 5. Đối chiếu với nhãn

Theo danh sách 1000 nhãn được train thì nhãn này là **cougar, puma, catamount, mountain lion, painter, panther, Felis concolor',**

[Danh sách nhãn](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)